In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [49]:
import numpy as np

In [2]:
host = 'fireteam108c-db.ctq6oauek6z5.us-east-2.rds.amazonaws.com:3306'
user = 'admin108c'
password = 'fireteam108c'
database = 'amazon_books'

In [3]:
def get_table_data(columns:list[str], table_name, uri:str, chunk_size:int=100000):
	engine = create_engine(uri)
	query = f"SELECT {", ".join(columns)} FROM {table_name}"

	with engine.connect() as connection:
		try:
			result = pd.read_sql_query(query, connection, chunksize=chunk_size)
			return pd.concat([chunk for chunk in result])
		except Exception as e:
			connection.rollback()
			connection.close()
			print("Error in reading data from database")
			print(e)

	

In [13]:
ratings_user_tagged = get_table_data(columns, "ratings_user_tagged", f"mysql+mysqldb://{user}:{password}@{host}/{database}")

In [14]:
ratings_user_tagged

,User_id,review/score,book_id
0,AVCGYZL8FQQTD,4.0,1882931173
1,A30TK6U7DNS82R,5.0,0826414346
2,A3UH4UZ4RSVO82,5.0,0826414346
3,A2MVUWT453QH61,4.0,0826414346
4,A22X4XUPKF66MR,4.0,0826414346
...,...,...,...
34968,AI1QNMVF2E3TN,5.0,B000NSLVCU
34969,AOFGOUMXLMVZS,4.0,B000NSLVCU
34970,A1SMUB9ASL5L9Y,4.0,B000NSLVCU
34971,A2AQMEKZKK5EE4,4.0,B000NSLVCU


In [15]:
res = { k: v.get(["book_id", "review/score"]) for k, v in ratings_user_tagged.groupby('User_id').filter(lambda x: len(x) > 10).groupby('User_id')}

In [20]:
reviewed_tagged_books = get_table_data(["book_id"], "reviewed_books", f"mysql+mysqldb://{user}:{password}@{host}/{database}")
reviewed_tagged_books

,book_id
0,1882931173
1,0826414346
2,0829814000
3,0595344550
4,0253338352
...,...
6706,0590482467
6707,0570047870
6708,B000OVF7JY
6709,1402508735


In [21]:
ratings_user_tagged["book_id"].unique().shape[0]

206711

In [22]:
reviewed_tagged_books

,book_id
0,1882931173
1,0826414346
2,0829814000
3,0595344550
4,0253338352
...,...
6706,0590482467
6707,0570047870
6708,B000OVF7JY
6709,1402508735


In [23]:
matrix = pd.DataFrame(index=res.keys(), columns = reviewed_tagged_books["book_id"].values)
matrix

,1882931173,0826414346,0829814000,0595344550,0253338352,0802841899,B0007FIF28,B000JINSBG,0895554224,0963923080,...,B0006E1KEK,0896102130,0812553861,0971556504,1418016985,0590482467,0570047870,B000OVF7JY,1402508735,0534400604
A01038432MVI9JXYTTK5T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A07587253PDB1WFIEPBPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100NGGXRQF0AQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100V1W0C8BWOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100YDQ9DIP06P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZY96LU53VWGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZYIHKBD3FZBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZ1E6SG4RAT7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZCSTUZ5EO4I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
for k, v in res.items():
	t = v.reset_index(drop=True).transpose()
	matrix.loc[k][t.loc["book_id"]] = t.loc["review/score"]
# 4 min 20 sec runtime


In [25]:
matrix

,1882931173,0826414346,0829814000,0595344550,0253338352,0802841899,B0007FIF28,B000JINSBG,0895554224,0963923080,...,B0006E1KEK,0896102130,0812553861,0971556504,1418016985,0590482467,0570047870,B000OVF7JY,1402508735,0534400604
A01038432MVI9JXYTTK5T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A07587253PDB1WFIEPBPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100NGGXRQF0AQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100V1W0C8BWOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100YDQ9DIP06P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZY96LU53VWGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZYIHKBD3FZBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZ1E6SG4RAT7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZCSTUZ5EO4I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
books_rating = pd.read_sql_table('books_rating', con=connection)
display(books_rating)

,index,Id,Title,review/helpfulness,review/score
0,47,0802841899,The Church of Christ: A Biblical Ecclesiology ...,74/81,5.0
1,48,0802841899,The Church of Christ: A Biblical Ecclesiology ...,2/3,5.0
2,49,0802841899,The Church of Christ: A Biblical Ecclesiology ...,2/3,4.0
3,50,0802841899,The Church of Christ: A Biblical Ecclesiology ...,3/5,4.0
4,195,0887402216,The Battleship Bismarck,9/9,3.0
...,...,...,...,...,...
174290,2998062,1585676187,Anthony Powell: A Life,9/10,5.0
174291,2998063,1585676187,Anthony Powell: A Life,13/16,4.0
174292,2998064,1585676187,Anthony Powell: A Life,8/10,1.0
174293,2998065,1585676187,Anthony Powell: A Life,2/2,4.0


In [17]:
books_ratings = pd.read_sql_table('books_rating_raw', con=connection)
books_ratings

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,NaN,None,None,14/19,4.0,937612800,Difficult,"This is an extremely difficult book to digest,..."
2999996,B000NSLVCU,The Idea of History,NaN,A1SMUB9ASL5L9Y,jafrank,1/1,4.0,1331683200,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2999997,B000NSLVCU,The Idea of History,NaN,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",0/0,4.0,1180224000,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2999998,B000NSLVCU,The Idea of History,NaN,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",1/11,5.0,1163030400,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


In [23]:
books_ratings = books_ratings.dropna()
books_ratings

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
10,0829814000,Wonderful Worship in Smaller Churches,19.40,AZ0IOBU20TBOP,Rev. Pamela Tinnin,8/10,5.0,991440000,Outstanding Resource for Small Church Pastors,"I just finished the book, &quot;Wonderful Wors..."
11,0829814000,Wonderful Worship in Smaller Churches,19.40,A373VVEU6Z9M0N,Dr. Terry W. Dorsett,1/1,5.0,1291766400,Small Churches CAN Have Wonderful Worship,Many small churches feel like they can not hav...
12,0829814000,Wonderful Worship in Smaller Churches,19.40,AGKGOH65VTRR4,"Cynthia L. Lajoy ""Cindy La Joy""",1/1,5.0,1248307200,Not Just for Pastors!,I just finished reading this amazing book and ...
13,0829814000,Wonderful Worship in Smaller Churches,19.40,A3OQWLU31BU1Y,Maxwell Grant,1/1,5.0,1222560000,Small church pastor? This is the book on worship,I hadn't been a small church pastor very long ...
14,0595344550,Whispers of the Wicked Saints,10.95,A3Q12RK71N74LB,Book Reader,7/11,1.0,1117065600,not good,I bought this book because I read some glowing...
...,...,...,...,...,...,...,...,...,...,...
2999953,0786182431,Very Bad Deaths: Library Edition,90.00,A1EC8SNPR56CLU,Denis Dube,0/0,4.0,1285804800,It's the way he writes it,"""Very Bad Death"" is a so so story, but the cha..."
2999954,0786182431,Very Bad Deaths: Library Edition,90.00,A33VKWCAV9QQKC,"Paige E. Steadman ""RuneEnigma""",0/0,5.0,1230249600,"Bad Deaths, Great Book!",Very Bad Deaths was a very great book! Spider ...
2999955,0786182431,Very Bad Deaths: Library Edition,90.00,A2PK3NTC9RMEF4,S. Potter,0/0,3.0,1179705600,Still read it.,Anything by Spider Robinson is worth reading. ...
2999956,0786182431,Very Bad Deaths: Library Edition,90.00,A2D0PY6HIGTYIT,"Adrian in Phoenix ""No Time for Fantasy""",5/8,5.0,1111276800,Not another Callahan story,Great novel! Easy & enjoyable to read straight...


In [26]:
books_ratings = books_ratings.drop_duplicates()

In [ ]:
books_ratings_tagged = pd.read_sql_table('books_rating', con=connection)

In [34]:
ratings_user_tagged = books_ratings[books_ratings["Id"].isin(books_ratings_tagged["Id"])].drop_duplicates().dropna()

In [ ]:
reviewed_tagged_books = pd.read_sql_table('reviewed_tagged_books', con=connection)

In [58]:
reviewed_tagged_books

,Title,review/average_score,ratingsCount,final_genre,book_id
0,"""Come to Me""",5.000000,6.0,Fiction,159467180X
1,"""Forget Not Love"": The Passion of Maximilian K...",5.000000,2.0,Art & Design,0898702755
2,"""Happiness Is Not My Companion"": The Life of G...",4.818182,1.0,Religion & Spirituality,0253339049
3,"""Life Was Never Meant to Be a Struggle""",4.411765,1.0,Religion & Spirituality,1561705357
4,"""Mom, Dad . . . I'm Pregnant"": When Your Daugh...",4.800000,3.0,Science & Technology,1576834824
...,...,...,...,...,...
9229,eBay: The Missing Manual,4.888889,2.0,Fiction,0596006446
9230,eMbedded Visual Basic: Windows CE and Pocket P...,4.000000,1.0,Literature & Poetry,0672322773
9231,mo+th issue 1,4.000000,17.0,Science & Technology,0941017680
9232,sendmail Desktop Reference (Pocket Reference),4.000000,3.0,Fiction,1565922786


In [59]:
ratings_user_tagged = pd.merge(books_ratings, reviewed_tagged_books.get(["Title","book_id"]), on='Title', how='inner')

In [66]:
ratings_user_tagged

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,book_id
0,0802841899,The Church of Christ: A Biblical Ecclesiology ...,25.97,ARI272XF8TOL4,Christopher J. Bray,74/81,5.0,955411200,Ecclesiological Milestone,With the publication of Everett Ferguson's boo...,0802841899
1,0802841899,The Church of Christ: A Biblical Ecclesiology ...,25.97,A36TPZSH8LBT1,haskell,2/3,5.0,1311465600,Early Christian development of the Church,Everett Ferguson approaches the subject of ear...,0802841899
2,0802841899,The Church of Christ: A Biblical Ecclesiology ...,25.97,ANX3DDV12ZRRU,GodsBreath.wordpress,2/3,4.0,1289952000,An Excellent Presentation of the Beliefs of th...,This book is a continual resource. It is so bi...,0802841899
3,0802841899,The Church of Christ: A Biblical Ecclesiology ...,25.97,A2H2LORTA5EZY2,Edward E. Howe,3/5,4.0,1266192000,Christ is Lord,This is a very useful and thorough text book. ...,0802841899
4,0887402216,The Battleship Bismarck,34.95,A30IUAABSEHEPI,Lawrence Duckles,9/9,3.0,1055980800,The Battleship Bismarck reviewed,This book is both a history and a photo album ...,0887402216
...,...,...,...,...,...,...,...,...,...,...,...
163997,1585676187,Anthony Powell: A Life,29.95,A117TCJ30OUVLK,hola,9/10,5.0,1199577600,A dance to the music of the author,"For any fan of the incredible series ""A Dance ...",1585676187
163998,1585676187,Anthony Powell: A Life,29.95,A30TK6U7DNS82R,Kevin Killian,13/16,4.0,1122163200,Fails to Bring Powell to Life,Michael Barber's immense biography is a benchm...,1585676187
163999,1585676187,Anthony Powell: A Life,29.95,A2HXQ64JSAXII8,Rampaging Hippogriff,8/10,1.0,1235174400,Dance to no music with two left feet,Having thoroughly enjoyed both my readings of ...,1585676187
164000,1585676187,Anthony Powell: A Life,29.95,A37P779U3Y116A,Historied,2/2,4.0,1281225600,Reasonable biography of a fascinating author,I suppose I am biased in that I really like An...,1585676187


In [82]:
res = { k: v.get(["book_id", "review/score"]) for k, v in ratings_user_tagged.groupby('User_id').filter(lambda x: len(x) > 10).groupby('User_id')}

In [106]:
matrix

,159467180X,0898702755,0253339049,1561705357,1576834824,0877880239,0966237420,0825424127,1572438479,B000FUFSM0,...,0940450836,0195121732,0595255833,0060740485,0060792167,0596006446,0672322773,0941017680,1565922786,063404169X
A106E1N0ZQ4D9W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A114YQ7ZT9Y1W5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A11B61QBGHLQDN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A11DCTGTPS7M0C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A11OBEH2ZCVAHX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AV9A3PYDX71JB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AX54G5AL870Q8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AXSH6O4OP5L4M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AYCJSA9HR7TKO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
matrix = pd.DataFrame(index=res.keys(), columns = reviewed_tagged_books["book_id"].values)

for k, v in res.items():
	t = v.reset_index(drop=True).transpose()
	matrix.loc[k][t.loc["book_id"]] = t.loc["review/score"]
matrix = matrix.fillna(0)
matrix.to_pickle("matrix.pkl")

In [120]:
matrix

,159467180X,0898702755,0253339049,1561705357,1576834824,0877880239,0966237420,0825424127,1572438479,B000FUFSM0,...,0940450836,0195121732,0595255833,0060740485,0060792167,0596006446,0672322773,0941017680,1565922786,063404169X
A106E1N0ZQ4D9W,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
A114YQ7ZT9Y1W5,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
A11B61QBGHLQDN,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
A11DCTGTPS7M0C,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
A11OBEH2ZCVAHX,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AV9A3PYDX71JB,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
AX54G5AL870Q8,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
AXSH6O4OP5L4M,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0
AYCJSA9HR7TKO,0,0,0.0,0.0,0,0,0,0,0,0,...,0,0.0,0,0.0,0.0,0.0,0,0,0,0


In [121]:
matrix.T.to_csv("matrix.csv.gzip", compression="gzip", encoding="utf-8", chunksize=1000)

In [33]:
matrix

,0826414346,0829814000,0963923080,0854968350,0918973031,0792391810,0789480662,0517150328,B000857LFE,1884734766,...,0765306832,B0002RQ1N6,B000BWUJLC,0679783334,0977070808,B0006E1KEK,0812553861,1418016985,0590482467,1402508735
A01038432MVI9JXYTTK5T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A07587253PDB1WFIEPBPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100NGGXRQF0AQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100V1W0C8BWOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100YDQ9DIP06P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZY96LU53VWGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZYIHKBD3FZBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZ1E6SG4RAT7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZCSTUZ5EO4I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
test = pd.read_csv("matrix.csv.gzip", compression='gzip', index_col=0)
test

,A01038432MVI9JXYTTK5T,A07587253PDB1WFIEPBPY,A100NGGXRQF0AQ,A100V1W0C8BWOL,A100YDQ9DIP06P,A100YHBWL4TR4D,A1012N48J0Z65N,A101446I5AWY0Z,A1016MYYF5QSTY,A101BVV4DR3G81,...,AZXFUIA3GYE0E,AZXGPM8EKSHE9,AZXQKAMHK35PA,AZY0QGEN37C9L,AZY8LGHVF8GMZ,AZY96LU53VWGO,AZYIHKBD3FZBI,AZZ1E6SG4RAT7,AZZCSTUZ5EO4I,AZZVZL4QEHEHO
0826414346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0829814000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0963923080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0854968350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0918973031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0006E1KEK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0812553861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418016985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0590482467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# matrix.replace(0, np.nan).isna().sum().sum()
test.notna().sum().sum()

548316